In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from data_process import build_vocab, batch_iter, sentence_to_index
from models import RNN

# RNN Operation: $h_t=tanh(W*[x_t;h_{t-1}] + b)$
<img src="https://cdn-images-1.medium.com/max/1600/1*NKhwsOYNUT5xU7Pyf6Znhg.png">

In [ ]:
batch_size = 2
input_size = 5
hidden_size = 3

In [ ]:
x_input = np.random.normal(size=(batch_size, input_size))
prev_state = np.random.normal(size=(batch_size, hidden_size))
w = np.random.normal(size=(input_size + hidden_size, hidden_size))
b = np.random.normal(size=(hidden_size))
print('x_input:', x_input, '\n')
print('prev_state:', prev_state)

In [ ]:
def RNN_numpy(x_input, previous_state, w, b):
    x = np.concatenate((x_input, previous_state), axis=1)
    state = np.tanh(np.add(np.matmul(x, w), b))
    output = state
    return output, state

In [ ]:
output, state = RNN_numpy(x_input, prev_state, w, b)
print('output:', output, '\n')
print('state:', state, '\n')
print('w:', w, '\n')
print('b:', b)

# RNN Operation using tensorflow

In [ ]:
tf.reset_default_graph()
x_input_tensor = tf.constant(x_input, dtype=tf.float32)
prev_state_tensor = tf.constant(prev_state, dtype=tf.float32)

In [ ]:
cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size, reuse=tf.AUTO_REUSE)
cell(x_input_tensor, prev_state_tensor)
init1 = tf.assign(cell.trainable_variables[0], w)
init2 = tf.assign(cell.trainable_variables[1], b)
sess = tf.Session()
sess.run([init1, init2])
print('x_input_tensor:', sess.run(x_input_tensor), '\n')
print('prev_state_tensor:', sess.run(prev_state_tensor), '\n')

In [ ]:
output_tensor, state_tensor = cell(inputs=x_input_tensor, state=prev_state_tensor)
print('output:', sess.run(output_tensor), '\n')
print('state:', sess.run(state_tensor), '\n')
for var in cell.trainable_variables:
    print(var.name, '\n', sess.run(var), '\n')

# Sentiment Analysis with RNN

In [ ]:
train = pd.read_csv('./data/train-5T.txt', delimiter='\t')
test = pd.read_csv('./data/test-1T.txt', delimiter='\t')

In [ ]:
X_train = train.document
Y_train = train.label
X_test = test.document
Y_test = test.label

In [ ]:
max_vocab = 50000
vocab, _, vocab_size = build_vocab(X_train, max_vocab)

In [ ]:
batches = batch_iter(list(zip(X_train, Y_train)), batch_size=64, num_epochs=15)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.reset_default_graph()
sess = tf.Session(config=config)
model = RNN(sess=sess, vocab_size=vocab_size, lr=1e-2)
avgLoss = []
x_test = sentence_to_index(X_test, vocab)

for step, batch in enumerate(batches):
    x_train, y_train = zip(*batch)
    x_train = sentence_to_index(x_train, vocab)
    l, _ = model.train(x_train, y_train)
    avgLoss.append(l)
    if step % 100 == 0:
        test_loss = model.get_loss(x_test, Y_test)
        print('batch:', '%04d' % step, 'train loss:', '%.5f' % np.mean(avgLoss), 'test loss:', '%.5f' % test_loss)
        test_acc = model.get_accuracy(x_test, Y_test)
        print('test accuracy:', '%.3f' % test_acc, '\n')
        avgLoss = []